In [1]:
import requests
import lxml
import time
import random
import os

import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [2]:
get1 = pd.read_csv('get3.csv')

In [3]:
#pub list
# driver = webdriver.Chrome(executable_path='C:\\chromedriver.exe')
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://scholar.google.com')
time.sleep(random.uniform(2, 3))

for j, personURL in enumerate(get1.ProfileLinks.values):
    driver.get(personURL)
    
    button = driver.find_element(By.XPATH,"//button[@id='gsc_bpf_more']")
    while True:
        if not button.is_enabled():
            break
        button.click()
        time.sleep(random.uniform(0.5, 1))
    
    text = driver.page_source
    soup = BeautifulSoup(text, 'html.parser')

    objects = soup.find_all('tr', {"class": "gsc_a_tr"})

    base = ['' for i in range(len(objects))]
    pubs = pd.DataFrame()
    pubs['titles'] = base
    pubs['authors'] = base
    pubs['date'] = base
    pubs['source'] = base
    pubs['descriptions'] = base
    pubs['citations'] = base

    for i, obj in enumerate(objects):
        pubs.loc[i, 'titles'] = obj.find_all('a')[0].text.replace('\xa0', ' ').replace('   ', ' ').replace('  ', ' ').replace('\u200b', '')
        time.sleep(random.uniform(0.75, 1.5))
        driver.find_element_by_link_text(pubs.loc[i, 'titles']).click()
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        p = soup.find_all('div', {"class": "gsc_oci_value"})
        if len(soup.find_all('div', {"class": "gsc_oci_value"})) >= 3:
            pubs.loc[i, 'authors'] = soup.find_all('div', {"class": "gsc_oci_value"})[0].text
            pubs.loc[i, 'date'] = soup.find_all('div', {"class": "gsc_oci_value"})[1].text
            pubs.loc[i, 'source'] = soup.find_all('div', {"class": "gsc_oci_field"})[2].text + ' ' + soup.find_all('div', {"class": "gsc_oci_value"})[2].text
        if soup.find_all('div', {"class": "gsh_csp"}):
            pubs.loc[i, 'descriptions'] = soup.find_all('div', {"class": "gsh_csp"})[0].text
        elif soup.find_all('div', {"class": "gsh_small"}):
            pubs.loc[i, 'descriptions'] = soup.find_all('div', {"class": "gsh_small"})[0].text
        if soup.find_all('div', {"style": "margin-bottom:1em"}):
            pubs.loc[i, 'citations'] = soup.find_all('div', {"style": "margin-bottom:1em"})[0].text[9:]
        time.sleep(random.uniform(1, 2))
        driver.back()
    pubs.to_csv(get1.loc[j, 'names'] + '.csv', index=False)


C:\Users\Simon\AppData\Local\Temp\ipykernel_12748\2851160434.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\Simon\AppData\Local\Temp\ipykernel_12748\2851160434.py:34: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  driver.find_element_by_link_text(pubs.loc[i, 'titles']).click()


KeyboardInterrupt: 

In [50]:
driver.close()